In [1]:
import findspark
findspark.init() # this must be executed before the below import
from pyspark.sql import SparkSession

In [2]:
import time

In [3]:
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

In [6]:
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/NORA_Test/merged`")
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/QdTree_Test/merged`")
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/NORA/merged`")

In [8]:
# print(df.count())
# print('= = = = = = = = = =')
# print(df.columns)
# print('= = = = = = = = = =')
# print(df.head(5))

In [40]:
# df.show()

+---------+--------+----+---+----+-----------------+-------------------+----+
|      _c0|     _c1| _c2|_c3| _c4|              _c5|                _c6| _c7|
+---------+--------+----+---+----+-----------------+-------------------+----+
|2421478.0|144996.0|18.0|3.0|43.0|         87762.57|               0.06|0.05|
| 929378.0|150001.0|16.0|2.0|42.0|          44142.0|               0.05|0.03|
| 943175.0|150003.0|18.0|1.0| 5.0|           5265.0|               0.01|0.04|
| 976262.0|150002.0|17.0|1.0|18.0|          18936.0|               0.01|0.01|
|2557920.0|150001.0|16.0|3.0| 2.0|           2102.0|               0.03|0.07|
|2558913.0|150001.0|16.0|3.0|45.0|          47295.0|               0.08|0.06|
|1875302.0|150000.0|15.0|3.0|15.0|          15750.0|               0.08|0.08|
|1120736.0|144991.0|13.0|1.0|47.0|         95691.53|               0.03|0.06|
| 440773.0|150001.0|16.0|2.0|10.0|          10510.0|               0.08|0.07|
|1234853.0|144995.0|17.0|5.0|41.0|83639.58999999998|            

In [22]:
# test query time NORA
start_time = time.time()
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/NORA_Test/merged` \
#                WHERE _c1 > 1000 AND _c1 < 9000 AND _c2 >= 114.4 AND _c2 <= 185.3")
df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/NORA_Test/merged` \
               WHERE _c1 > 1000 AND _c1 < 9000 AND _c2 >= 114.4 AND _c2 <= 185.3").collect()
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/NORA/merged` \
#                WHERE _c2 >= 114.4 AND _c2 <= 185.3")
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/NORA/merged` \
#                WHERE _c2 >= 114.4 AND _c2 <= 185.3").collect()
end_time = time.time()
print('execution time: ', end_time - start_time)

# execution time:  0.08401131629943848    # 4 partition without collect
# execution time:  0.47371959686279297    # 4 partition with collect

# execution time:  0.11887288093566895    # 1K+ partition without collect
# execution time:  1.857546329498291      # 1K+ partition with collect

execution time:  0.4454672336578369


In [23]:
# test query time Qd-Tree
start_time = time.time()
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/QdTree_Test/merged` \
#                WHERE _c1 > 1000 AND _c1 < 9000 AND _c2 >= 114.4 AND _c2 <= 185.3  ")
df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/QdTree_Test/merged` \
               WHERE _c1 > 1000 AND _c1 < 9000 AND _c2 >= 114.4 AND _c2 <= 185.3  ").collect()
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/QdTree/merged` \
#                WHERE _c2 >= 114.4 AND _c2 <= 185.3")
# df = spark.sql("SELECT * FROM parquet.`hdfs://localhost:9000/user/cloudray/QdTree/merged` \
#                WHERE _c2 >= 114.4 AND _c2 <= 185.3").collect()
end_time = time.time()
print('execution time: ', end_time - start_time)

# execution time:  0.08320879936218262 # 1 partition without collect
# execution time:  1.5957744121551514  # 1 partition with collect

# execution time:  0.07                # 56 partition without collect
# execution time:  1.309511423110962   # 56 partition with collect

execution time:  1.179133415222168


In [ ]:
# conclusion:
# 1. Each partition should not be too small, or the meta data calculation time will domainate the cost
# 2. The number of partitions (between 2 methods) should not vary too much, or the meta calculation time
#    could take a large portion